# PINN

This notebook is dedicated to the training of the Physics-Inspired-Neural-Network.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
import time
import torch
import pandas as pd

from pathlib import Path
from PINN import train_PINN_ADAM

## Used variables

In [3]:
short_rate_params = pd.read_csv('/Users/goshabor/NTNU/9.Semester/TMA4500 – Fordypningsprosjekt/Code/CIRPlusPlusResults/short_rate_params.csv')

r0 = float(short_rate_params['r0'].iloc[0])
Theta_r = short_rate_params.iloc[0, 2:6].astype(float).to_list()
Svensson_params = short_rate_params.iloc[0, 6:].astype(float).to_list()

print(f'Theta_r = {Theta_r}')
print(f'r0 = {r0}')

Theta_r = [0.10070984, 0.01024475, 0.03245518, 0.02578739]
r0 = 0.0191672169145129


In [4]:
# filepaths to store models
filepath_histories = '/Users/goshabor/NTNU/9.Semester/TMA4500 – Fordypningsprosjekt/Code/PINNResults/TrainingHistories/'
filepath_models = '/Users/goshabor/NTNU/9.Semester/TMA4500 – Fordypningsprosjekt/Code/PINNResults/TrainedModels/'

histories_directory = Path(filepath_histories).expanduser()
histories_directory.mkdir(parents=True, exist_ok=True)

model_directory = Path(filepath_models).expanduser()
model_directory.mkdir(parents=True, exist_ok=True)

## PINN Training using ADAM

We now perform a weighted training of the PINN model.

In [5]:
#bounds_S
strike_price = 50
factor = 3
S_min = 0.1*strike_price; S_max = factor*strike_price
bounds_S = [S_min, S_max]

#bounds r
r_min = 0.01; r_max = 0.05
bounds_r = [r_min, r_max]

#bounds_nu
nu_min = 0.03**2; nu_max = 0.5**2
kappa_nu_min = 5e-1; kappa_nu_max = 5.0
theta_nu_min = 0.05**2; theta_nu_max = 0.5**2
sigma_nu_min = 1e-1; sigma_nu_max = 1.0

bounds_nu = [[nu_min, nu_max], [kappa_nu_min, kappa_nu_max], [theta_nu_min, theta_nu_max], [sigma_nu_min, sigma_nu_max]]

#bounds_rho
rho_min = -1.0; rho_max = 1.0
bounds_rho = [rho_min, rho_max]

#bounds_time
T_min = 0.0; T_max = 3.0
bounds_time = [T_min, T_max]

input_dim = 9
hidden_dim_arr = [512]
num_layers_arr = [5]

batch_boundary = 5000
batch_interior = 20000 

#training related
epochs_per_phase = [500, 1000, 1500, 2000, 2000] #1-500, 501-1500, 1501-3000, 3001-5000, 5001-7000
learning_rates = [5e-3, 2e-3, 1e-3, 1e-4, 1e-5]

results = [] #to store filepaths and model losses

#boundary_loss_weights = [weights_interior, weights_lower, weights_upper, weights_terminal, weights_MC]
boundary_loss_weights = [1, 1, 1, 1]

### --- Weighted PINN Training ---
for hidden_dim in hidden_dim_arr:
    for num_layers in num_layers_arr:
        print(f'------------------------------------------------ Training {int(num_layers-1)}x{hidden_dim} weighted model ------------------------------------------------')
        start_time = time.perf_counter()

        PINN_params = [input_dim, hidden_dim, num_layers, batch_boundary, batch_interior]

        #PINN training
        model, history = train_PINN_ADAM(strike_price=strike_price, bounds_S=bounds_S, Theta_r=Theta_r, bounds_r=bounds_r, bounds_nu=bounds_nu, bounds_rho=bounds_rho, bounds_time=bounds_time, Svensson_params=Svensson_params, PINN_params=PINN_params, 
                                epochs_per_phase=epochs_per_phase, learning_rates=learning_rates, boundary_loss_weights=boundary_loss_weights, print_every=100)
        
        #saving histories and model for later evaluation
        tag = f'{int(num_layers-1)}x{hidden_dim}'

        histories_path = histories_directory / f'{tag}_{factor}x_2.xlsx'
        ckpt_path = model_directory / f'{tag}_{factor}x_2.pt'

        cols = ['phase', 'epoch', 'total loss', 'interior loss', 'loss lower', 'loss upper', 'terminal']
        pd.DataFrame(history, columns=cols).to_excel(f'{histories_path}', index=False)

        ckpt = {
            'model_state': model.state_dict(),
            'config': {
                'PINN_params': PINN_params,
                'bounds_S': bounds_S,
                'bounds_r': bounds_r,
                'bounds_nu': bounds_nu,
                'bounds_rho': bounds_rho,
                'bounds_time': bounds_time,
                'Theta_r': Theta_r,
                'Svensson_params': Svensson_params,
                'strike_price': strike_price,
                'boundary_loss_weights': boundary_loss_weights,
                'dtype': str(next(model.parameters()).dtype), 
                'epochs': epochs_per_phase,
                'lr': learning_rates
            }
        }
        torch.save(ckpt, ckpt_path)

        final_total = float(history[-1][2]) if len(history) else float('nan')
        results.append({
            'tag': f'{int(num_layers-1)}x{hidden_dim}',
            'final_total_loss': final_total,
            'ckpt_path': ckpt_path,
            'history_csv': histories_path
        })

        #freeing memory to avoid memory leak
        del model, history
        gc.collect()
        
        if torch.backends.mps.is_available():
            torch.mps.empty_cache()
        elif torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        end_time = time.perf_counter()
        elapsed_time_minutes = (start_time-end_time)/60

        print(f'Model saved as {tag}_{factor}x_2.pt | Execution time: {abs(elapsed_time_minutes):.4f} minutes')
        print(f'------------------------------------------ Finished training {tag} weighted model ------------------------------------------')

------------------------------------------------ Training 4x512 weighted model ------------------------------------------------
Phase 1/5 | lr=0.005 | 1/500 | Total loss=2537.568 | Interior loss=2.511 | Loss lower=4.174 | Loss upper=0.988 | Loss terminal=2529.895


KeyboardInterrupt: 